In [1]:
!pip install pdf2image
!pip install smbprotocol

!pip install --upgrade google-cloud-aiplatform

!pip install google-cloud-core==1.6.0
!pip install google-cloud-documentai==1.2.0


  Using cached google_cloud_core-1.6.0-py2.py3-none-any.whl (28 kB)
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 2.3.2
    Uninstalling google-cloud-core-2.3.2:
      Successfully uninstalled google-cloud-core-2.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 2.9.0 requires google-cloud-core<3.0dev,>=2.3.0, but you have google-cloud-core 1.6.0 which is incompatible.
kfp 1.8.22 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.90.0 which is incompatible.


In [2]:
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (20.09.0-3.1+deb11u1).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [4]:
%%capture

import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [5]:
from google.cloud import documentai_v1 as documentai
from PIL import Image, ImageDraw

import os
import pandas as pd
from IPython.display import display, Image
from pdf2image import convert_from_path, convert_from_bytes

In [6]:
PROJECT_ID = "playground-350714"
PROJECT_NUMBER = "340976817408"
LOCATION = "eu"  # Format is 'us' or 'eu'
PROCESSOR_ID = "fe9efa246ac573db"  # Create processor in Cloud Console

In [7]:
from google.api_core.client_options import ClientOptions
client_options = {"api_endpoint": "{}-documentai.googleapis.com".format(LOCATION)}
#client_options = ClientOptions(api_endpoint=f"{LOCATION}-documentai.googleapis.com")


client = documentai.DocumentProcessorServiceClient(client_options=client_options)

def process_document_sample(file_path, confidence_threshold=0.6):
    # Instantiates a client

    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id
    # You must create new processors in the Cloud Console first
    name = f"projects/{PROJECT_NUMBER}/locations/{LOCATION}/processors/{PROCESSOR_ID}"

    with open(file_path, "rb") as image:
        image_content = image.read()

    # Read the file into memory
    document = {"content": image_content, "mime_type": "application/pdf"}
    # document = {"content": image_content, "mime_type": "image/tiff"}

    # Configure the process request
    request = {"name": name, "raw_document": document}

    # Recognizes text entities in the PDF document
    result = client.process_document(request=request)
    document = result.document
    entities = document.entities
    print("Document processing complete.\n\n")

    # For a full list of Document object attributes, please reference this page: https://googleapis.dev/python/documentai/latest/_modules/google/cloud/documentai_v1beta3/types/document.html#Document
    types = []
    values = []
    confidence = []

    # Grab each key/value pair and their corresponding confidence scores.
    for entity in entities:
        if entity.confidence > confidence_threshold:
            types.append(entity.type_)
            values.append(entity.mention_text)
            confidence.append(round(entity.confidence,4))

    # Create a Pandas Dataframe to print the values in tabular format.
    df = pd.DataFrame({'Type': types, 'Value': values, 'Confidence': confidence})
    display(df)

    # if result.human_review_operation:
    #     print ("Triggered HITL long running operation: {}".format(result.human_review_operation))

    return document, df

In [23]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai  # type: ignore
# TODO(developer): Uncomment these variables before running the sample.
project_id = '340976817408'
location = 'eu' # Format is 'us' or 'eu'
processor_id = 'fe9efa246ac573db' #  Create processor before running sample
processor_version_id = 'pretrained-ocr-v1.0-2020-09-23' # Processor version to use
file_path = 'asteroid_split.pdf'
mime_type = 'application/pdf' # Refer to https://cloud.google.com/document-ai/docs/file-types for supported file types
# field_mask = "text,entities,pages.pageNumber"  # Optional. The fields to return in the Document object.


def process_document_processor_version_sample(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version_id: str,
    file_path: str,
    mime_type: str,
) -> None:
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the processor version
    # e.g. projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}
    name = f"projects/{PROJECT_NUMBER}/locations/{LOCATION}/processors/{PROCESSOR_ID}"


    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Load Binary Data into Document AI RawDocument Object
    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)

    # Configure the process request
    request = documentai.ProcessRequest(
        name=name, raw_document=raw_document
    )

    result = client.process_document(request=request)

    # For a full list of Document object attributes, please reference this page:
    # https://cloud.google.com/python/docs/reference/documentai/latest/google.cloud.documentai_v1.types.Document
    document = result.document

    # Read the text recognition output from the processor
    print("The document contains the following text:")
    print(document.text)


In [8]:
from smbclient import link, open_file, register_session, remove, rename, stat
from smbclient import listdir, mkdir,rmdir, scandir
from smbclient.path import isdir



register_session("10.164.0.2", username='share', password='W@achtwoord2023!')

#for filename in listdir(r"\\10.164.0.2\share"):
#    print(filename)

    
    
    
    
with open_file(r"\10.164.0.2\share\AAE\astdelux.pdf", mode="rb") as fd:
    file_bytes = fd.read()
    
#images = convert_from_bytes(file_bytes)
#display(images[0])

with open("test.pdf","wb") as binary_file:

    binary_file.write(file_bytes)

In [12]:
doc, df = process_document_sample(f"moo.pdf")


Document processing complete.




,Type,Value,Confidence


In [11]:
print(df)

Empty DataFrame
Columns: [Type, Value, Confidence]
Index: []


In [24]:
process_document_processor_version_sample(project_id,location,processor_id, processor_version_id, file_path, mime_type)

The document contains the following text:
ASTEROIDS
Operation, Maintenance
and Service Manual
Complete with Illustrated Parts Lists
EmuMovies
O
TM-143
3rd printing
a
ATARI
Ⓡ
GAME SERIAL NUMBER LOCATION
Your game's serial number is located on the outside rear of the game. The
same number is also stamped on the chassis of the TV monitor, Game PCB
and Regulator/Audio PCB. Please mention this number whenever calling your
distributor for service.
GAME SERIAL #
IDENTIFICATION TAG
TV MONITOR
REGULATOR/AUDIO PCB
GAME PCB
ASTEROIDS
Published by:
ATARI INC
1265 Borregas Avenue
P. O. Box 427
Sunnyvale, California 94086
Operation, Maintenance
and Service Manual
Complete with Illustrated Parts Lists
Copyright © 1979 by Atari, Inc. All rights reserved
No part of this publication may be reproduced by any mechanical,
photographic, or electronic process, or in the form of a phonographic re-
cording, nor may it be stored in a retrieval system, transmitted, or other-
wise copied for public or private use